In [1]:
import networkx as nx;
#import matplotlib.pyplot as plt
import pandas as pd;
import gurobipy as gp;
from gurobipy import GRB;
import csv;
import sys;
import time;
from datetime import datetime;
import math;
import random;
import itertools;
import numpy as np;

In [2]:
def pessimistic_DelayedRelax (network, budget, T_Limit, summaryName):
    
    start_time = time.time(); 
    networkCSV = network+'.csv';

    # Reading network file
    with open(networkCSV, newline='') as f:
        reader = csv.reader(f);
        row1 = next(reader);
        s = int(row1[0]);             # Source node
        t = int(row1[1]);             # Sink node
        calA_level = int(row1[2]);    # Level of special arcs
    
        G = nx.DiGraph();
        data = pd.read_csv(networkCSV, skiprows=1, header=None);
        n_edge = len(data.index+1);
    
        for i in range(n_edge): 
            G.add_edge(data.iat[i,0], data.iat[i,1], capacity= data.iat[i,2], 
                    cost=data.iat[i,3], special=data.iat[i,4], level=data.iat[i,5]);  
    
    
    A = 0;
    U = G.edges[t,s]['capacity'];
    for i,j in G.edges:
        if G.edges[i,j]['special'] == 1:
            A = A + 1;
            
    
    ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ###
    
    
    ### Optimization Model ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    M = A;
    Md = A;
    Mt = A;
    
    instance = 'LogFile_'+network+'_b'+str(budget)+'.txt';
    
    tau = range(math.ceil(math.log(U))+1);
    
    model = gp.Model("DelayedRelax"); 
    
    eta = model.addVar(vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
    gamma = model.addVars(G.edges, vtype=GRB.BINARY);
    xBar = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    x = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    z = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = 1); 
    
    alpha = model.addVars(G.nodes, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = GRB.INFINITY); 
    theta = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
    pi = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb =-GRB.INFINITY, ub=0);
    omega = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY);
    delta = model.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub=0);          
    v = model.addVars(G.edges, vtype=GRB.CONTINUOUS, lb = 0, ub = GRB.INFINITY); 
    muTau = model.addVars(tau, vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);
    mu = model.addVar(vtype=GRB.CONTINUOUS, lb = -GRB.INFINITY, ub = 0);  
    lamb = model.addVars(tau, vtype=GRB.BINARY);
    
    
    model.addConstr(gp.quicksum(gamma[i,j]*G.edges[i,j]['cost'] for i,j in G.edges) <= budget);   
    
    model.addConstrs(gp.quicksum(xBar[j,i] for i in G.successors(j))-
                     gp.quicksum(xBar[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
    model.addConstrs(gp.quicksum(x[j,i] for i in G.successors(j))-
                     gp.quicksum(x[i,j] for i in G.predecessors(j)) == 0 for j in G.nodes);
            
    model.addConstr(xBar[t,s] == gp.quicksum(pow(2, tau[u])*lamb[u] for u in tau));
    
    for i,j in G.edges:
        model.addConstr(xBar[i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0);
        model.addConstr(x[i,j] - (1-gamma[i,j])*G.edges[i,j]['capacity'] <= 0); 
        model.addConstr(v[i,j] <= theta[i,j]);    
        model.addConstr(v[i,j] >= theta[i,j] - Mt*gamma[i,j]);
        model.addConstr(v[i,j] <= Mt*(1-gamma[i,j]));
        if G.edges[i,j]['special'] == 1:
            model.addConstr(alpha[i] - alpha[j] + theta[i,j] + pi[i,j]*G.edges[i,j]['special'] >= 0); 
            model.addConstr(x[i,j] - (1/M)*z[i,j]*G.edges[i,j]['special'] >= 0);
            model.addConstr(omega[i,j]*G.edges[i,j]['special'] - (1/M)*pi[i,j]*G.edges[i,j]['special'] >= 1);
        elif i!=t:
            model.addConstr(alpha[i] - alpha[j] + theta[i,j] >= 0); 
    
    model.addConstr(x[t,s] >= xBar[t,s]);      
    model.addConstr(alpha[t] - alpha[s] + theta[t,s] + delta >= 0);                                    
    model.addConstr(mu == gp.quicksum(pow(2, tau[u])*muTau[u] for u in tau));
    
    for u in tau:
        model.addConstr(muTau[u] >= delta);
        model.addConstr(muTau[u] >= -Md*lamb[u]);
        model.addConstr(muTau[u] <= delta - Md*(lamb[u] - 1));
    
    model.addConstr(eta >= gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges)); 
    
    model.addConstr(gp.quicksum(z[i,j]*G.edges[i,j]['special'] for i,j in G.edges)>=
                             gp.quicksum(G.edges[i,j]['capacity']*v[i,j] for i,j in G.edges)+
                             gp.quicksum(omega[i,j]*G.edges[i,j]['special'] for i,j in G.edges) + mu);
    
    
    model.setObjective(eta, GRB.MINIMIZE);
    
    model.setParam("IntegralityFocus",1);
    #model.setParam("MIPFocus", 3);
    #model.setParam('NodefileStart', 0.5)    # Memore Issues
    #model.setParam("NumericFocus",2);
    model.setParam('TimeLimit', T_Limit); 
    model.update();
    
    model.setParam("LogToConsole", 0)
    model.setParam("OutputFlag", 1);
    model.setParam("LogFile", instance);
    model.optimize();
    
    status = model.status;
    
    end_time = time.time();
    run_time = round(end_time - start_time, 2);
    
    now = datetime.now();
    
    obj = round(model.objVal);
    LB = model.objBound;
    sol = A - obj;
    OptGap = round(model.MIPGap, 2);
    
    
        ### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ####
    
    ### Print to the screen and to result file #######################
    
    file = open('DelayedRelax_'+network+'_b'+str(budget)+'.txt', "w");
    file.write('Instance: %s, budget %g \n' %(network, budget));
    file.write('Instance executed on: %s \n\n' %now.strftime("%c"));
    
    file.write('Number of Nodes: %g \n' % (t+1));
    file.write('Number of Arcs: %g \n' % G.number_of_edges());
    file.write('Level of special set: %g \n' % calA_level);
    file.write('Budget: %g' % budget +'\n\n');
    
    
    print('\n')
    print('Instance: %s, budget %g \n' %(network, budget));
    if status == 2:
        print('Status: Optimal');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);
        
        file.write('Status: Optimal\n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);
        
    elif status == 9:
        print('Status: Time Limit');
        print('Number of special arcs without flow: %g' % sol);
        print('Number of special arcs with flow: %g' % obj);     
        print('Optimality Gap: %g' % OptGap);
        
        file.write('Status: Time Limit \n');
        file.write('Number of special arcs without flow: %g \n' % sol);
        file.write('Number of special arcs with flow: %g \n' % obj);     
        file.write('Optimality Gap: %g \n' % OptGap);
              
    else:
        print('Status: %g' % status);
        file.write('Status: %g \n' % status);
    
    print('\nFlow: %g' %x[t,s].x);
    file.write('Flow: %g \n' %x[t,s].x);
    
    print('run time: %g sec \n' %run_time);
    file.write('run time: %g sec \n' %run_time);
    
    file.write('Level 0 Capacities: %g \n\n ' %G.edges[s,1]['capacity']);
    
    print('Intediction:')
    for i,j in G.edges:
        if gamma[i,j].x > 0.0001:
            print("arc (%g,%g), gamma = %g " %(i,j, gamma[i,j].x));
    print('\n');
    
    
    lev1 = 0;
    lev2 = 0;
    lev3 = 0;
    other_level = 0;
    
    file.write('\nInterdiction plan: \n');
    
    key1 = False;
    key2 = False;
    key3 = False;
    
    for i, j in G.edges: 
        if gamma[i,j].x > 0.0001:
            if G.edges[i,j]["level"] == 1:
                if key1 == False:
                    file.write("Level 1:" +'\n');
                    key1 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev1 += 1;
            elif G.edges[i,j]["level"] == 2:
                if key2 == False:
                    file.write('Level 2: \n');
                    key2 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev2 += 1;
            elif G.edges[i,j]["level"] == 3:
                if key3 == False:
                    file.write('Level 3: \n');
                    key3 = True;
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                lev3 += 1;
            else:
                file.write('Other Levels: \n');
                file.write("arc (%g,%g), gamma = %g \n" %(i,j, gamma[i,j].x));
                other_level += 1;
                
    file.write('\nSpecial arcs with flow : \n');    
    for i, j in G.edges:
        if G.edges[i,j]['special'] == 1:
            if x[i,j].x > 0.00001:
                file.write('Arc (%s,%s), Flow: %f \n' %(i,j, x[i,j].x));
    
    
    
    file.write('\n');
    file.close();
    
    rowFields = [network, budget, calA_level, (t+1), G.number_of_edges(), OptGap, round(obj), x[t,s].x,
                 lev1, lev2, lev3, other_level, run_time];
    
    with open(summaryName, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile);
        csvwriter.writerow(rowFields);
        csvfile.close();

    

In [3]:

n_Networks = 5;
Budget = [0, 3, 6];


T_Limit = 30;

summaryName = "DelayedRelax_Summary.csv"
file_summary = open(summaryName, "w");
file_summary.write('Instance\t, Budget\t, Cal_A_Level\t, Nodes\t, Arcs\t, OptGap\t,');
file_summary.write('Obj_value\t, Flow\t, Level_1\t, Level_2\t, Level_3\t, Other_Levels\t, Run_Time\n');
file_summary.close();

for n in range(1, n_Networks+1):
    network = 'Net'+str(n);
    
    for budget in Budget:
        pessimistic_DelayedRelax(network, budget, T_Limit, summaryName);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-28
Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 30


Instance: Net1, budget 0 

Status: Optimal
Number of special arcs without flow: 0
Number of special arcs with flow: 16

Flow: 1
run time: 0.04 sec 

Intediction:


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 30


Instance: Net1, budget 3 

Status: Optimal
Number of special arcs without flow: 3
Number of special arcs with flow: 13

Flow: 126
run time: 0.16 sec 

Intediction:
arc (2,8), gamma = 1 
arc (2,9), gamma = 1 
arc (5,10), gamma = 1 


Set parameter IntegralityFocus to value 1
Set parameter TimeLimit to value 30


Instance: Net1, budget 6 

Status: Optimal
Number of special arcs without flow: 6
Number of special arcs with flow: 10

Flow: 138.25
run time: 4.19 sec 

Intediction:
arc (4,10), gamma = 1 
arc (5,10), gamma = 1 
arc (8,11), gamma = 1 
arc (8,13), gamma = 1 
arc (8,14), g